**First exercise: count the number of lines in Python for each file**

In [31]:
def countLines(fileName:String) : Unit = {
    val bookings_rdd = sc.textFile(fileName)
    //val nb_lines = bookings_rdd.map(line => 1).reduce(_+_)
    println("nb lines in file" + fileName + " : " + bookings_rdd.count())
    }
    
//TODO bz2 files
countLines("downloads/bookings.csv")
//countLines("downloads/archives.csv.bz2")


nb lines in filedownloads/bookings.csv : 10000011


**Second exercise: top 10 arrival airports in the world in 2013 (using the bookings file)**

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


In [7]:
import org.apache.spark.sql.{SQLContext, Row, DataFrame}

def topAirportsSQL(fileName:String, nb_top:Int) : Unit = {    
    val sqlContext = new org.apache.spark.sql.SQLContext(sc)  
    import sqlContext.implicits._
    var bookings_df = sqlContext.load("com.databricks.spark.csv", Map("path" -> fileName,
                                                                      "header"->"true", 
                                                                      "charset"->"UTF8",
                                                                      "delimiter"->"^", 
                                                                      "nullvalue" -> "NULL",
                                                                      "inferschema" -> "true",
                                                                      "treatEmptyValuesAsNulls" -> "true",
                                                                      "parserLib" -> "univocity",
                                                                      "ignoreTrailingWhiteSpace" ->"true",
                                                                      "codec" ->"bzip2"
                                                                     ))                                            
   // bookings_df.printSchema()
    val topAirports = bookings_df.select("arr_port", "pax")
    .na.fill(0)
                                 .groupBy("arr_port")
                                 .sum()
                                 .sort($"sum(pax)".desc)


    topAirports.show(nb_top)
    }

topAirportsSQL("downloads/bookings.csv", 10)

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 63 in stage 16.0 failed 1 times, most recent failure: Lost task 63.0 in stage 16.0 (TID 609, localhost): java.lang.NumberFormatException: null
	at java.lang.Integer.parseInt(Integer.java:542)
	at java.lang.Integer.parseInt(Integer.java:615)
	at scala.collection.immutable.StringLike$class.toInt(StringLike.scala:229)
	at scala.collection.immutable.StringOps.toInt(StringOps.scala:31)
	at com.databricks.spark.csv.util.TypeCast$.castTo(TypeCast.scala:61)
	at com.databricks.spark.csv.CsvRelation$$anonfun$buildScan$6.apply(CsvRelation.scala:194)
	at com.databricks.spark.csv.CsvRelation$$anonfun$buildScan$6.apply(CsvRelation.scala:173)
	at scala.collection.Iterator$$anon$13.hasNext(Iterator.scala:371)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:327)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:327)
	at org.apache.spark.sql.execution.aggregate.TungstenAggregationIterator.proc

**Third exercise: plot the monthly number of searches for flights arriving at Málaga, Madrid or Barcelona**

For the arriving airport, you can use the Destination column in the searches file. Plot a curve for Málaga, another one for Madrid, and another one for Barcelona, in the same figure. Bonus point: Solving this problem using pandas (instead of any other approach)
